In [37]:
import vectorbt as vbt
import numpy as np
import numpy.ma as ma
import pandas as pd
import inspect
import datetime
import talib
from IPython.display import display, HTML


vbt.settings.data['alpaca']['key_id'] = 'PK2XNKDSJH4PVPCYDK8E'
vbt.settings.data['alpaca']['secret_key'] = 'f5cD1kbR3p5RRfgvNgDiQ09qRpi4LCYmPZprl3KM'

In [38]:
symbol = 'AAPL'
data = vbt.AlpacaData.download(symbol,
                               start='2023-3-05',
                               end='2023-3-16',
                               # limit=1000,
                               timeframe='15m').get()

inspect.signature(data.vbt.ohlcv.plot)
data.vbt.ohlcv.plot(plot_type='candlestick')


AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'close': array([152.5 , 152.28, 152.35, ..., 152.9 , 152.92, 152.8 ]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([152.86, 152.51, 152.36, ..., 153.  , 152.99, 152.93]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([152.05, 152.27, 152.22, ..., 152.82, 152.83, 152.71]),
              'name': 'Candlestick',
              'open': array([152.05, 152.5 , 152.27, ..., 152.83, 152.91, 152.92]),
              'type': 'candlestick',
              'uid': '3441db9f-e81e-48ab-b35c-6ef45e90e60b',
              'x': array([datetime.datetime(2023, 3, 6, 9, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 6, 9, 15, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 6, 9, 30, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 3, 15, 20, 15, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 15, 20, 30, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 15, 20, 45, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'yaxis': 'y'},
             {'marker': {'color': array(['#1b9e76', '#d95f02', '#1b9e76', ..., '#1b9e76', '#1b9e76', '#d95f02'],
                                        dtype=object),
                         'line': {'width': 0}},
              'name': 'Volume',
              'opacity': 0.5,
              'type': 'bar',
              'uid': 'e4cea973-f0d7-4eb2-b283-612348263680',
              'x': array([datetime.datetime(2023, 3, 6, 9, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 6, 9, 15, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 6, 9, 30, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 3, 15, 20, 15, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 15, 20, 30, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 15, 20, 45, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x2',
              'y': array([44163., 11807., 16450., ..., 86602., 51670., 31512.]),
              'yaxis': 'y2'}],
    'layout': {'bargap': 0,
               'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'showlegend': True,
               'template': '...',
               'width': 700,
               'xaxis': {'anchor': 'y',
                         'domain': [0.0, 1.0],
                         'matches': 'x2',
                         'rangeslider': {'visible': False},
                         'showgrid': True,
                         'showticklabels': False},
               'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0], 'showgrid': True},
               'yaxis': {'anchor': 'x', 'domain': [0.3, 1.0], 'showgrid': True},
               'yaxis2': {'anchor': 'x2', 'domain': [0.0, 0.3], 'showgrid': True}}
})

In [39]:
def is_hammer(close, high, low, start, ratio, girth):
    girth *= ratio
    fib = (high-low)*ratio
    lowerbound = high-fib

    is_hammer = np.where((start > lowerbound) & (((high-low)*girth) <= (close - start)), 1, 0)
    return is_hammer

    #if open > lowerbound and open-close > 0:
    #    return close.index[-1]
    #else:
    #    return None

IS_HAMMER = vbt.IndicatorFactory(
    class_name = "is_hammer",
    short_name = "is_hammer",
    input_names = ["close", "high", "low", "start"],
    output_names = ["value"],
    param_names=['ratio', 'girth'],
).from_apply_func(
    is_hammer
)

hammer = IS_HAMMER.run(data['Close'], data['High'], data['Low'], data['Open'], 0.32, .1)
data['is_hammer'] = hammer.value

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)

#print(hammer.value)

#hammer = talib.CDLHAMMER(data['Open'], data['High'], data['Low'], #data['Close'])
#print(hammer[hammer==100])

,Open,High,Low,Close,Volume,is_hammer
timestamp,,,,,,
2023-03-06 09:00:00+00:00,152.0500,152.8600,152.0500,152.5000,44163.0,0
2023-03-06 09:15:00+00:00,152.5000,152.5100,152.2700,152.2800,11807.0,0
2023-03-06 09:30:00+00:00,152.2700,152.3600,152.2200,152.3500,16450.0,0
2023-03-06 09:45:00+00:00,152.2900,152.3000,152.1700,152.2700,6658.0,0
2023-03-06 10:00:00+00:00,152.3200,152.5500,152.3100,152.5500,9904.0,0
2023-03-06 10:15:00+00:00,152.5500,152.5700,152.4700,152.4700,4219.0,0
2023-03-06 10:30:00+00:00,152.4500,152.5000,152.3000,152.3200,13059.0,0
2023-03-06 10:45:00+00:00,152.3200,152.8700,152.3200,152.8100,16019.0,0
2023-03-06 11:00:00+00:00,152.7600,152.8800,152.6800,152.8500,6702.0,0


In [40]:
def hammer_buy(hammer_filter, high, close, low, timeperiod_atr, risk_to_reward, atr_multiplier, n_shift):
    #calculating ATR
    atr = talib.ATR(high, low, close, timeperiod_atr)

    #If 3 candles after hammer closes higher than hammer high
    high = ma.masked_array(high, mask=np.logical_not(hammer_filter), fill_value=np.inf).filled()
    high = np.roll(high, n_shift)
    high[:n_shift] = np.inf

    #calculating buy, stop loss, take profit
    stop_loss = low-atr*atr_multiplier
    take_profit = close+(atr_multiplier*atr)*risk_to_reward

    return np.where(close >= high, 1, 0, close, stop_loss, take_profit),
HAMMER_BUY = vbt.IndicatorFactory(
    class_name = "hammer_buy",
    short_name = "hammer_buy",
    input_names = ['hammer', 'high', 'close', 'low', 'timeperiod_atr'],
    output_names = ['buy_times'],
    param_names=['n_shift', 'risk_to_reward', 'atr_multiplier'],
).from_apply_func(hammer_buy, timeperiod_atr = 14, risk_to_reward = 1.71, atr_multiplier = 1, n_shift = 3)

buy_times = HAMMER_BUY.run(hammer.value, data['High'], data['Close'], data['Low'], timeperiod_atr=14, risk_to_reward = 1.71, atr_multiplier = 1, n_shift=3)
data['buy'] = buy_times.buy_times
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)

TypeError: only size-1 arrays can be converted to Python scalars